This tutorial shows the potential of using CO2 capture resources in Macro such as Natural Gas Power CCS, SMR H2 CCS. Natural Gas Power CCS and SMR H2 CCS works by specifying a CO2 capture rate (for example 0.9) of the total CO2 emitted by natural gas used in producing power and H2 which effectively reduces the total emissions by 90%. There are a total of 2 CO2 node types: 1) Captured CO2 sink node, where its input is for captured CO2 from Natural Gas Power CCS and SMR H2 CCS, 2) Emission CO2 sink node to account for emission by fossil sources.

In [1]:
using Pkg
Pkg.activate(".")

  Activating environment at `~/Documents/GitHub/Macro/tutorials/Project.toml`


In [2]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [3]:
T = 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2_Captured=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [4]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(Electricity => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2_Captured => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), NaturalGas => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1)), PeriodLength = 8760)

In [5]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;
ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas

ngcc_ccs_inv_cost = 66400;# $/MW
ngcc_ccs_fom_cost = 12287.0;# $/MW
ngcc_ccs_vom_cost = 3.65; #$/MW
ngcc_ccs_capsize = 250.0;
ngcc_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas
ngcc_ccs_CO2_captured_rate = 0.9; #Tonnes of CO2 captured/Tonness of CO2 produced by fuel

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MW of H2
electrolyzer_vom_cost = 0.0;

smr_inv_cost = 1033333/H2_MWh;# $/MW of H2
smr_fom_cost = 20500/H2_MWh;# $/MW of H2
smr_vom_cost = 0.0; #$/MW of H2
smr_capsize = 20*H2_MWh;
smr_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas

smr_ccs_inv_cost = 1133333/H2_MWh;# $/MW of H2
smr_ccs_fom_cost = 21500/H2_MWh;# $/MW of H2
smr_ccs_vom_cost = 0.0; #$/MW of H2
smr_ccs_capsize = 20*H2_MWh;
smr_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas
smr_ccs_CO2_captured_rate = 0.9; #Tonnes of CO2 captured/Tons of CO2 produced by fuel


In [6]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [7]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [8]:
solar_pv = Resource{Electricity}(;
    id = :Solar_PV,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    capacity_factor = solar_capacity_factor,
    can_expand = true, 
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = solar_inv_cost,
    fixed_om_cost = solar_fom_cost,
    constraints = [Macro.CapacityConstraint()]
)

Resource{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :Solar_PV, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.3834, 0.2594, 0.078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, true, false, 85300.0, 18760.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

We can also assigne a battery storage to the node:

In [9]:
battery = SymmetricStorage{Electricity}(;
    id = :battery,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    can_expand = true,
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = battery_inv_cost,
    investment_cost_storage = battery_inv_cost_storage,
    fixed_om_cost = battery_fom_cost,
    fixed_om_cost_storage = battery_fom_cost_storage,
    variable_om_cost = battery_vom_cost,
    variable_om_cost_storage = battery_vom_cost_storage,
    efficiency_injection = battery_eff_down,
    efficiency_withdrawal = battery_eff_up,
    constraints = [Macro.CapacityConstraint(),Macro.StorageCapacityConstraint()]
)

SymmetricStorage{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :battery, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0.0, Inf, 0.0, Inf, 0.0, 0.0, true, false, 19584.0, 22494.0, 0.0, 4895.0, 5622.0, 0.0, 0.15, 0.15, 0.0, 0.92, 0.92, 0.0, 1.0, 10.0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing), Macro.StorageCapacityConstraint(missing, missing, missing)])

We also consider a hydrogen demand node:

In [10]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [11]:
ng_node = SourceNode{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    price = ng_fuel_price,
    constraints = [Macro.DemandBalanceConstraint()]
)

SourceNode{NaturalGas}(:NG_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264  …  14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We also define a CO2 emission sink node:

In [12]:
co2_node = SinkNode{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
constraints = [Macro.DemandBalanceConstraint()]
#### Note that if we are modeling a CO2 cap, we could add such a constraint to this sink
)


SinkNode{CO2}(:CO2_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Float64[], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We define an atmospheric CO2 source node to provide DAC with CO2 from the atmosphere to capture:

We define a captured CO2 sink node to account for the inventory of captured CO2 (input from CO2 capture, output to SNG or potential storage):

In [13]:
co2_captured_node = SinkNode{CO2_Captured}(;
id = Symbol("CO2_Captured_node"),
time_interval = time_interval(CO2_Captured),
subperiods = subperiods(CO2_Captured),
constraints = [Macro.DemandBalanceConstraint()]
)


SinkNode{CO2_Captured}(:CO2_Captured_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Float64[], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

This transformation has 3 edges: each edge carries a specific commodity flow

In [14]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
number_of_stoichiometry_balances = 2,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_heatrate,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_fuel_CO2]
))

push!(ngcc.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 65400.0, 10287.0, 3.55, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{N

We are now ready to define a natural gas plant with CCS! Note that we require three stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas $\times$ (1-CCS rate) = CO2

Fuel_CO2 $\times$ NaturalGas $\times$ CCS rate = CO2_Captured

This transformation has 4 edges: each edge carries a specific commodity flow

In [15]:
ngcc_ccs = Transformation{NaturalGasPowerCCS}(;
id = :NGCC_CCS,
time_interval = time_interval(Electricity),
number_of_stoichiometry_balances = 3,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc_ccs.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc_ccs,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_ccs_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_ccs_heatrate,0.0,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_ccs_inv_cost,
fixed_om_cost = ngcc_ccs_fom_cost,
variable_om_cost = ngcc_ccs_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_ccs_fuel_CO2*(1-ngcc_ccs_CO2_captured_rate),ngcc_ccs_fuel_CO2*ngcc_ccs_CO2_captured_rate]
))

push!(ngcc_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(ngcc_ccs.TEdges,TEdge{CO2_Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPowerCCS}(:NGCC_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 66400.0, 12287.0, 3.65, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [16]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                number_of_stoichiometry_balances = 1,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(electrolyzer.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [1.0],
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(electrolyzer.TEdges,TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = [electrolyzer_efficiency]
))

2-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 66.66, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0], 0.0, Inf, 0.0, 61006.090609060906, 915.0915091509152, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConst

We are now ready to define a natural gas hydrogen plant or SMR similar to a natural gas power plant

In [17]:
smr = Transformation{NaturalGasHydrogen}(;
                id = :SMR,
                time_interval = time_interval(Hydrogen),
                number_of_stoichiometry_balances = 2,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_heatrate,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_inv_cost,
    fixed_om_cost = smr_fom_cost,
    variable_om_cost = smr_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_fuel_CO2]
))

push!(smr.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(:SMR, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 31003.090309030904, 615.0615061506151, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, An

Now, let's add a SMR hydrogen plant with CCS, similar to a natural gas power plant with CCS

In [18]:
smr_ccs = Transformation{NaturalGasHydrogenCCS}(;
                id = :SMR_CCS,
                time_interval = time_interval(Hydrogen),
                number_of_stoichiometry_balances = 3,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr_ccs.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_ccs_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_ccs_heatrate,0.0,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_ccs_inv_cost,
    fixed_om_cost = smr_ccs_fom_cost,
    variable_om_cost = smr_ccs_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_ccs_fuel_CO2*(1-smr_ccs_CO2_captured_rate),smr_ccs_fuel_CO2*smr_ccs_CO2_captured_rate]
))

push!(smr_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(smr_ccs.TEdges,TEdge{CO2_Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogenCCS}(:SMR_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 34003.390339033904, 645.0645064506451, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), 

Now that we have implemented different technologies, we can create the JuMP model.

In [19]:
components = [solar_pv; battery; ngcc; ngcc_ccs; electrolyzer; smr; smr_ccs];
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;components];

In [20]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [21]:
add_planning_variables!.(components,model);

In [22]:
add_operation_variables!.(system, model);

In [23]:
add_all_model_constraints!.(system, model);

In [24]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [25]:
println(ngcc.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] = 0


In [26]:
println(ngcc.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [27]:
println(ngcc_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] = 0


In [28]:
println(ngcc_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [29]:
println(ngcc_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_Captured_CO2_Captured[5] = 0


In [30]:
println(electrolyzer.constraints[1].constraint_ref[1,5])

vFLOW_Hydrogen_H2[5] - 0.7406666666666667 vFLOW_Electricity_E[5] = 0


In [31]:
println(smr.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] = 0


In [32]:
println(smr.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [33]:
println(smr_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] = 0


In [34]:
println(smr_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [35]:
println(smr_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_Captured_CO2_Captured[5] = 0


In [36]:
using HiGHS

In [37]:
Macro.set_optimizer(model,HiGHS.Optimizer)

In [38]:
Macro.optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
91831 rows, 74319 cols, 240453 nonzeros
91831 rows, 74319 cols, 240453 nonzeros
Presolve : Reductions: rows 91831(-144699); columns 74319(-144706); elements 240453(-350724)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 17520(5.77987e+07) 0s
      59243     3.5072693726e+09 Pr: 0(0); Du: 0(9.35764e-12) 3s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 59243
Objective value     :  3.5072693726e+09
HiGHS run time      :          3.42


The installed electrolyzer capacity is:

In [39]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

0.0

The installed SMR capacity is

In [40]:
Macro.value(Macro.capacity(smr.TEdges[1]))

1039.5627

The installed SMR CCS capacity is

In [41]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

0.0

The installed solar capacity in MW is:

In [42]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity in MW is:

In [43]:
Macro.value(Macro.capacity(battery))

100.0

The installed NGCC capacity in MW is:

In [44]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

16617.0

The installed NGCC CCS capacity in MW is:

In [45]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [46]:
base_emissions  = Macro.value(sum(Macro.outflow(co2_node)[t] for t in time_interval(CO2)))

3.429744069856643e7

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [47]:
base_CO2_captured  = Macro.value(sum(Macro.outflow(co2_captured_node)[t] for t in time_interval(CO2_Captured)))

0.0

Lets add a low CO2 cap:

In [48]:
Macro.@constraint(model,CO2Cap,sum(Macro.outflow(co2_node)[t] for t in time_interval(CO2)) <= 1000000);

In [49]:
Macro.optimize!(model)

Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     3.5073991633e+09 Pr: 1(6.65949e+07) 0s
       1282     3.5073991684e+09 Pr: 1047(3.30581e+08); Du: 0(0.000155499) 5s
       2569     3.5073991684e+09 Pr: 2121(3.28751e+08); Du: 0(9.19659e-05) 10s
       3937     3.5073991684e+09 Pr: 3242(3.26829e+08); Du: 0(2.31829e-05) 16s
       5381     3.5161055389e+09 Pr: 5424(3.69324e+08); Du: 0(5.33598e-07) 22s
       6974     3.5161085785e+09 Pr: 5639(3.74601e+08); Du: 0(6.27746e-07) 29s
       8567     3.5161105175e+09 Pr: 6000(3.83002e+08); Du: 0(9.39347e-07) 36s
      10167     3.5161123509e+09 Pr: 6583(3.7754e+08); Du: 0(1.32341e-06) 43s
      11718     3.5161145740e+09 Pr: 7216(3.06406e+08); Du: 0(9.99106e-07) 49s
      13180     3.5161467738e+09 Pr: 9296(1.09067e+09); Du: 0(9.77039e-07) 55s
      14480     3.5179744606e+09 Pr: 13787(1.30308e+10); Du: 0(8.97733e-07) 61s
      16048     3

Now, the electrolyzer capacity is:

In [50]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

884.2449

The SMR capacity is:

In [51]:
Macro.value(Macro.capacity(smr.TEdges[1]))

-0.0

The SMR CCS capacity is:

In [52]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

878.2455

Now the solar capacity is:

In [53]:
Macro.value(Macro.capacity(solar_pv))

42923.03856856197

The installed battery capacity is:

In [54]:
Macro.value(Macro.capacity(battery))

8720.917181138502

The installed NG capacity is:

In [55]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

-0.0

The installed NG CCS capacity is:

In [56]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

6980.082818861499

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [57]:
Macro.value(sum(Macro.outflow(co2_node)[t] for t in time_interval(CO2)))

999999.9999999948

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [58]:
Macro.value(sum(Macro.outflow(co2_captured_node)[t] for t in time_interval(CO2_Captured)))

8.999999999999998e6

We see that in a low CO2 emission case, Power and H2 CCS are deployed along with PV and electrolyzers

However, if we use a net-zero CO2 cap:

In [59]:
Macro.@constraint(model,CO2CapZero,sum(Macro.outflow(co2_node)[t] for t in time_interval(CO2)) <= 0);

In [60]:
Macro.optimize!(model)

Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     8.1017450691e+09 Pr: 1(2e+06) 74s
       1675     1.1399430472e+10 Pr: 20638(3.55686e+08); Du: 0(2.05923e-06) 82s
       2969     1.1713892059e+10 Pr: 22893(2.46992e+08); Du: 0(4.00798e-06) 88s
       4033     1.1949499955e+10 Pr: 25689(9.03233e+08); Du: 0(2.9676e-06) 94s
       5159     1.2366648174e+10 Pr: 5953(1.48791e+08); Du: 0(1.46943e-06) 99s
       6419     1.2643257386e+10 Pr: 5528(3.21923e+08); Du: 0(4.74763e-07) 104s
       7984     1.3331977950e+10 Pr: 17181(3.04381e+08); Du: 0(6.2311e-08) 111s
       9639     1.4499765748e+10 Pr: 15011(1.8273e+08) 118s
      10993     1.5595120758e+10 Pr: 1192(8.40264e+06) 123s
      12620     1.8089306563e+10 Pr: 11553(7.38555e+07); Du: 0(1.02049e-07) 129s
      14564     2.0904497230e+10 Pr: 9146(6.0158e+06) 135s
      15089     2.1558249169e+10 Pr: 0(0); Du: 0(6.9129e-11) 137s
Model  

Now, the electrolyzer capacity is:

In [61]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

1039.5626999999656

The SMR capacity is:

In [62]:
Macro.value(Macro.capacity(smr.TEdges[1]))

-0.0

The SMR CCS capacity is:

In [63]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

-4.092726157978176e-12

Now the solar capacity is:

In [64]:
Macro.value(Macro.capacity(solar_pv))

119184.68700864779

The installed battery capacity is:

In [65]:
Macro.value(Macro.capacity(battery))

16372.849999999959

The installed NG capacity is:

In [66]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

-0.0

The installed NG CCS capacity is:

In [67]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [68]:
Macro.value(sum(Macro.outflow(co2_node)[t] for t in time_interval(CO2)))

1.1600571351653243e-11

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [69]:
Macro.value(sum(Macro.outflow(co2_captured_node)[t] for t in time_interval(CO2_Captured)))

-2.957457047703989e-11

We see that in a net-zero emission case, only PV power and electrolyzer H2 are deployed.